<div style="background-color: #1e1e2e; color: #cdd6f4;"><center><br><h1>KDAG Task 1 - Langton's Ant Simulation</h1>

<hr>

<h3>This notebook was written by <b>Rohan Kumar Sah</b> - 24ME10134</h3>


Each cell contains its explanation right below. This was fun to code!</center><br></div>


In [7]:
import pygame
import sys
from pygame.locals import *
import random
import numpy as np

h, w = 1024, 768
cell_size = 8
rows, cols = h // cell_size, w // cell_size
lineColour = (0, 255, 0) 
antColour = (255, 0, 0)

pygame.init()
screen = pygame.display.set_mode((h, w))
myfont = pygame.font.SysFont("Roboto", 36)
clock = pygame.time.Clock()
FPS = 60

Matrix = [[0 for _ in range(cols)] for _ in range(rows)]
Pheromones = np.array([[0 for _ in range(cols)] for _ in range(rows)])
Factors = np.array([[0 for _ in range(cols)] for _ in range(rows)])

speed_multiplier = 5

We begin with importing all the necessary libraries required and initialising lots of stuff.  
- The dimensions for the window and cell size 
- Standard PyGame initialisation and font for the step counter to be rendered 
- A 2-D Matrix having 0 for white ⚪ ; and 1 for black ⚫ corresponding to each cell in the grid 
- A Factors Matrix for inculcating decay, I will go into its working later in the code 

Feel free to tweak the constants to get different results.




In [8]:
class Ant:
    def __init__(self, x, y, sign, direction=None):
        self.x, self.y, self.sign  = x, y, int(sign)
        self.direction = direction if direction is not None else random.choice([0,1,2,3])

    def move(self):
        Prob = 0
        factor = Factors[self.y][self.x]
        selfProb = factor * 0.8
        crossProb = factor * 0.2
        if Pheromones[self.y][self.x]==self.sign:
            Prob = random.choices(["straight", "standard"], weights=[selfProb, 1-selfProb])[0]

        if Pheromones[self.y][self.x]!=self.sign and Pheromones[self.y][self.x]!=0:
            Prob = random.choices(["straight", "standard"], weights=[crossProb, 1-crossProb])[0]

        Matrix[self.y][self.x] = 1 - Matrix[self.y][self.x]

        if Prob == "straight":
            Pheromones[self.y][self.x] = self.sign
            Factors[self.y][self.x] = 1

            if self.direction == 0: self.x -= 1
            elif self.direction == 1: self.y += 1
            elif self.direction == 2: self.x += 1
            elif self.direction == 3: self.y -= 1
            
        else:
            Pheromones[self.y][self.x] = self.sign
            Factors[self.y][self.x] = 1

            self.direction = (self.direction + 1) % 4 if Matrix[self.y][self.x] == 1 else (self.direction - 1) % 4

            if self.direction == 0: self.x -= 1
            elif self.direction == 1: self.y += 1
            elif self.direction == 2: self.x += 1
            elif self.direction == 3: self.y -= 1

        
        self.x %= cols
        self.y %= rows

Ant1 = Ant(rows // 3, 1*cols // 4, 1)
Ant2 = Ant(rows // 3 , 3*cols // 4, 2)

steps = 0
last_move_time = pygame.time.get_ticks()

We define a class __Ant__ with the following attributes
- __x__ and __y__ coordinates of the ant
- __sign__ of the ant which is basically the pheromone indicator for the ant to be stored in __Pheromones__ Matrix
- __direction__ is randomly initialised from one of `[0,1,2,3]` which denote the four directions

<div class="alert alert-block alert-warning">
All instances using the default value will share the same randomly chosen <b>direction</b> if we calculate direction in the function signature. So, we put it inside the constructor. This ensures that each Ant instance gets its own random direction when no direction is provided.
</div>

The `move()` function works as follows
- We first calculate probability taking the factor $ \in \{0, 0.2, 0.4, 0.6, 0.8, 1\} $ into account for both Self and Cross Recognition
- `Random.choices()` is used to calculate the ant's movement depending on the Pheromone on the cell
- The ant proceeds to move leaving behind its pheromone and refreshing the factor to 1

We wrap around the grid to accomodate for when the ant reaches the boundaries

At last, we initialise Ant1 and Ant2 and the steps and the last_move_time


In [9]:
running = True
while running:
    screen.fill((255, 255, 255))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    current_time = pygame.time.get_ticks()

    if current_time - last_move_time >= 100 / speed_multiplier:
        Ant1.move()
        Ant2.move()
        steps += 1
        Factors = np.maximum(0,Factors-0.2)
        last_move_time = current_time

    for y in range(rows):
        for x in range(cols):
            color = (0, 0, 0) if Matrix[y][x] == 1 else (255, 255, 255)
            pygame.draw.rect(screen, color, Rect((y * cell_size, x * cell_size), (cell_size, cell_size)))

    pygame.draw.rect(screen, antColour, Rect((Ant1.y * cell_size, Ant1.x * cell_size), (cell_size, cell_size)))
    pygame.draw.rect(screen, antColour, Rect((Ant2.y * cell_size, Ant2.x * cell_size), (cell_size, cell_size)))

    for i in range(rows + 1):
        pygame.draw.line(screen, lineColour, (i * cell_size, 0), (i * cell_size, w))
    for i in range(cols + 1):
        pygame.draw.line(screen, lineColour, (0, i * cell_size), (h, i * cell_size))

    label = myfont.render("Steps : " + str(steps), True, (0, 0, 255))
    screen.blit(label, (10, 10))

    pygame.display.update()
    clock.tick(FPS)

pygame.quit()

We are done with all the necessary preparations and now begin with the infinite simulation which runs as follows

1. Clear the screen
2. Setup quit mechanism
3. Execute code block only if a certain amount of time has passed since last move
4. Move ants, increment steps and decay the factors by $ 0.2 $ (All factors keep decrementing by 0.2 in every step to reach zero by 5th step until refreshed)
5. Draw cells, ants and grid lines
6. Update the step counter on the screen
7. Update the display


<center><h1>The End</h1><hr></center>